In [1]:
%load_ext autoreload
%autoreload 2

import logging
import requests
import json
import os
import time
import pandas as pd
import numpy as np

wd = os.path.abspath("__file__").replace("/__file__", "").replace("notebooks", "")
os.chdir(wd)

import requests
from datetime import datetime, timedelta, date
from logging.handlers import TimedRotatingFileHandler

from src.utils import get_jinja_yaml_conf, create_db_engine, Postgres_connect, send_message_to_discord
from src.processing import *
from src.api import *

now = datetime.now()

In [2]:
conf = get_jinja_yaml_conf('./conf/webhook.yml', './conf/logging.yml', './conf/data.yml')
end_date = datetime.now().date() 

stream = logging.StreamHandler()
stream.setLevel(logging.DEBUG)
# logger 설정
logger = logging.getLogger('main')
logging.basicConfig(level=eval(conf['logging']['level']),
    format=conf['logging']['format'],
    handlers = [TimedRotatingFileHandler(filename =  conf['logging']['file_name'],
                                when=conf['logging']['when'],
                                interval=conf['logging']['interval'],
                                backupCount=conf['logging']['backupCount']), #logging.StreamHandler()
                                   stream]
                )


In [3]:
# Only for notebooks
import re

os.environ['_ts'] = datetime.astimezone(datetime.now()).strftime('%Y-%m-%d %H:%M:%S %z')

with open('./conf/credentials', "r") as file:
    # 각 라인 읽기
    for line in file:
        # 주석(#) 또는 빈 줄은 무시
        if line.strip() == '' or line.startswith('#'):
            continue

        # 각 라인을 '='를 기준으로 key와 value로 분리
        key, value = line.strip().split('=', 1)

        # $ENV 형식의 환경변수가 있을 경우 해당 값을 가져와서 설정
        env_var_pattern = re.compile(r'\$(\w+)')
        matches = env_var_pattern.findall(value)
        for match in matches:
            value = value.replace(f"${match}", os.environ.get(match, "")).replace('"', '')

        # 환경변수로 설정
        os.environ[key] = value

In [4]:
engine = create_db_engine(os.environ)
postgres_conn = Postgres_connect(engine)
api = Coingecko()

2024-04-09 16:42:08,142 (utils.py 75) INFO ::: Connect to 172.20.10.3. DB_NAME is coins
2024-04-09 16:42:08,142 (api.py 23) INFO ::: ### Coingecko main api is initialized! ###


In [26]:
conf['data']['daily_coin_info']['max_pages'] = 1

In [27]:
processing_daily_coin_info(postgres_conn, api, conf['data']['daily_coin_info'], conf['webhook'])

2024-03-31 13:56:16,625 (api.py 79) INFO ::: Get daily coin info (1/3)
2024-03-31 13:56:16,698 (api.py 48) INFO ::: Exceeded api call rate limit. API call now tries: 1/3; sleep 600 secs.
2024-03-31 13:56:16,698 (api.py 49) INFO ::: {"status":{"error_code":429,"error_message":"You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits."}}


KeyboardInterrupt: 

In [ ]:
detail_url = conf_nfts_list['url'], params = conf_nfts_list['params']

In [5]:
conf['data']['nfts_list']['params']

{'per_page': 250}

In [7]:
api.get_data(conf['data']['nfts_list']['url'], params = conf['data']['nfts_list']['params']).drop_duplicates()

,id,contract_address,name,asset_platform_id,symbol,_ts
0,squiggly,0x36F379400DE6c6BCDF4408B282F8b685c56adc60,Squiggly,ethereum,~~,2024-04-09 16:42:07.533427+09:00
1,voxelglyph,0xa94161fbe69e08ff5a36dfafa61bdf29dd2fb928,Voxelglyph,ethereum,#,2024-04-09 16:42:07.533427+09:00
2,autoglyphs,0xd4e4078ca3495de5b1d4db434bebc5a986197782,Autoglyphs,ethereum,☵,2024-04-09 16:42:07.533427+09:00
3,spacepunksclub,0x45db714f24f5a313569c41683047f1d49e78ba07,SpacePunksClub,ethereum,⚇,2024-04-09 16:42:07.533427+09:00
4,meebits,0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7,Meebits,ethereum,⚇,2024-04-09 16:42:07.533427+09:00
...,...,...,...,...,...,...
245,the-sandbox-assets,0xa342f5d851e866e18ff98f351f2c6637f4478db5,The Sandbox ASSETS,ethereum,ASSET,2024-04-09 16:42:07.533427+09:00
246,assetdash-vanta,,AssetDash Vanta,solana,ASSETDASH VANTA,2024-04-09 16:42:07.533427+09:00
247,the-association,0x9b8b9c7c02be0bd0aa4d669bf6a1f6003424c6dc,The Association,ethereum,ASSOC,2024-04-09 16:42:07.533427+09:00
248,astral-babes,29b65f763b01fc7304e9f2325c6c02b47e128009e58102...,Astral Babes,ordinals,ASTRAL-BABES,2024-04-09 16:42:07.533427+09:00
